In [1]:
from bs4 import BeautifulSoup
import requests
from csv import writer
import time
import random
from lxml import etree as et
import urllib.request
import re
import numpy as np

In [2]:
pages_url = []
url = "https://batdongsan.vn/ban-nha/p"
for i in range(50):
    page_url = url + str(i+301+50)
    pages_url.append(page_url)

In [3]:
def download_html(url):
    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')
    response.close()
    return html

In [4]:
def get_house_url(soup_page_url):           # link_one_page_no_change
    source = download_html(soup_page_url)            #dowload_link_pages_
    soup_page_url = BeautifulSoup(source, 'html.parser')
    house_url_pages = soup_page_url.find('div',{'class':'datalist'}).findAll('div', {'class':'name'})
    for i in range(len(house_url_pages)):
        link = house_url_pages[i].find('a')['href']
        house_url_pages[i]=link
    return house_url_pages                  # link_each_house_on_one_page_co_change


In [5]:
def scrap (location, user, price, infor, title, time):   # title_type_string
    
    data={}
    try:
        data['user'] = user.get_text()
    except:
        data['user'] = None
    try: 
        data['user_link'] = user.find('a')['href']
    except:
        data['user_link'] = None
    data['time'] = time
    try: 
        data['Tinh/TP'] = location[2].get_text()
        data['Quan/Huyen'] = location[3].get_text()
    except:
        data['Tinh/TP'] = None
        data['Quan/Huyen'] = None
    for i in infor:
        data[i.find('strong').text] = None
        data[i.find('strong').text] = str(re.search(r'[\d.]+',i.find(text=True, recursive=False).strip()))
        if (i.find('strong').text != 'Diện tích' or i.find('strong').text != 'Phòng ngủ' or i.find('strong').text != 'Phòng WC'):
            data[i.find('strong').text] = i.find(text=True, recursive=False).strip()
        
    data['price'] = None
    price = price.contents[0].strip()
    if price != 'Thỏa thuận':
        price_ = re.match(r'^([\d.,]+)\s*(\D+)$',price)
        unit = price_.group(2) 
        if unit == 'triệu':
            data['price'] =  float(price_.group(1)) / 1000
        else:
            if unit == 'tỷ':
                data['price'] =  price_.group(1)
    else:
        matches = re.findall(r'(\d+([\.,]\d+)?)\s*(tỷ|tỉ|ty)(\d{0,3})?', title, re.IGNORECASE)
        if matches:
            prices = [match[0] for match in matches]
            data['price'] = prices[-1]
    data['Tầng'] = None
    matches = re.findall(r'(\d+)\s*(tầng)', title, re.IGNORECASE)
    if matches:
        floors = [match[0] for match in matches]
        data['Tầng'] = floors[0]
    else:
        matches = re.findall(r'(\d+)\s*(lầu)', title, re.IGNORECASE)
        if matches:
            floors = [match[0] for match in matches]
            data['Tầng'] = int(floors[0]) + 1
    

    return data


In [6]:
def extract_house_info(text):
    info = {}
    info['giá'] = 0
    matches = re.findall(r'(\d+([\.,]\d+)*)\s*(tỷ|tỉ|ty)(\d{0,3})*', text, re.IGNORECASE)
    if matches:
        prices = [match[0] for match in matches]
        info['giá'] = prices[-1]
    if info['giá'] == 0:
        matches = re.findall(r'(\d+([\.,]\d+)*)\s*(tr|trieu|triệu)(\d{0,3})*', text, re.IGNORECASE)
        if matches:
            prices = [match[0] for match in matches]
            info['giá'] = float(prices[-1]) / 1000
    match = re.search(r'((DT:|diện tích:)\s*[\d.]+)|([\d.]\s*+(m2)+)', text, re.IGNORECASE)
    if match:
        info['Diện tích'] = match.group(1)

    # Lấy thông tin về số phòng ngủ
    match = re.search(r'(\d+)\s*(pn|phòng ngủ)', text, re.IGNORECASE)
    if match:
        info['Phòng ngủ'] = str(match.group(1))

    # Lấy thông tin về số phòng WC
    match = re.search(r'(\d+)\s*(wc|phòng vệ sinh)', text, re.IGNORECASE)
    if match:
        info['Phòng WC'] = str(match.group(1))

    # Lấy thông tin về số tầng
    match = re.search(r'(\d+)\s*(tầng)', text, re.IGNORECASE)
    if match:
        info['Số tầng'] = match.group(1)
    else:
        info['Số tầng'] = 0
    # Lấy thông tin về số lầu
    match = re.search(r'(\d+)\s*(lầu)', text, re.IGNORECASE)
    if match:
        info['Số lầu'] = match.group(1)
    else:
        info['Số lầu'] = 0
    match = re.search(r'(\d+)\s*(trệt)', text, re.IGNORECASE)
    if match:
        info['trệt'] = match.group(1)
    else:
        info['trệt'] = 0
    if info['Số tầng'] == 0:
        if info['Số lầu'] == 0 and info['trệt'] != 0 :
            info['Số tầng'] = int(info['trệt']) + 1
        if info['trệt'] == 0 and info['Số lầu'] != 0:
            info['Số tầng'] = int(info['Số lầu']) + 1
        if info['trệt'] != 0 and info['Số lầu'] != 0:
            info['Số tầng'] = int(info['Số lầu']) + int(info['trệt'])

    # Kiểm tra sự có mặt của sân
    match = re.search(r'([\w]+\s*(sân))|((sân)\s[\w]+)', text, re.IGNORECASE)
    if match:
        info['Có sân'] = match.group()

    match = re.search(r'([\w\s]+\s*(bệnh viện))|((bệnh viện)\s[\w\s]+)', text, re.IGNORECASE)
    if match:
        info['Gần bệnh viện'] = match.group()

    match = re.search(r'([\w\s]+\s*(chợ|siêu thị))|((chợ|siêu thị)\s[\w\s]+)', text, re.IGNORECASE)
    if match:
        info['Gần chợ'] = match.group()

    match = re.search(r'([\w\s]+\s*(trường))|((trường)\s[\w\s]+)', text, re.IGNORECASE)
    if match:
        info['Gần trường'] = match.group()

    match = re.search(r'((dân trí)\s[\w\s]+)|([\w\s]+\s*(dân trí))', text, re.IGNORECASE)
    if match:
        info['Dân trí'] = match.group()

    match = re.search(r'((an ninh)\s[\w\s]+)|([\w\s]+\s*(an ninh))', text, re.IGNORECASE)
    if match:
        info['An ninh'] = match.group()

    match = re.search(r'([\w\s]+\s*(kinh doanh|buôn bán))|((kinh doanh|buôn bán)\s[\w\s]+)', text, re.IGNORECASE)
    if match:
        info['Kinh doanh'] = match.group()

    match = re.search(r'((vị trí:|địa điểm:|khu vục:số|đường|quận|huyện|tỉnh|thành phố|tp)[\s\w,]+)', text, re.IGNORECASE)
    if match:
        info['Địa điểm'] = match.group()
    
    return info

In [7]:
def merge_libraries(library1, library2):
    merged_library = {}

    # Gộp thư viện 1
    for key, value in library1.items():
        merged_library[key] = value

    # Gộp thư viện 2
    for key, value in library2.items():
        merged_library[key] = value

    return merged_library

In [8]:
def scrape_this(link_each_house_pages):            # link_each_house_on_one_page
    source_house = download_html(link_each_house_pages) 
    soup_house = BeautifulSoup(source_house,'html.parser')  # change_link
    data1={}
    data2={}
    data=[]
    try:
        location = soup_house.find('ul',{'class':'uk-breadcrumb'}).findAll('li')
    except:
        location=''
    try:
        user = soup_house.find('div', {'class':'name'})
    except:
        user=''
    try:
        price = soup_house.find('strong', class_='price')
    except:
        price=''
    try:
        info = soup_house.find('div', {'class':'body'}).findAll('ul',{'class':'uk-list'})
    except:
        info=''
    try:
        time = info[1].findAll('li')
        info = info[0].findAll('li')
    except:
        time = info[0]
    try:
        title = soup_house.find('h1',{'class':'uk-panel-title'}).get_text()
    except:
        title=''
    try:
        content = soup_house.find('div',{'class':'content'}).get_text()
    except:
        content=''
  
    data1 = scrap(location, user, price, info, title, time)
    data2 = extract_house_info(content)
    data.append(merge_libraries(data1, data2))
    return data

In [ ]:
data=[]
print(len(pages_url))
for i in range(len(pages_url)):                    #link_pages
    print(i+301+50)
    #input_link_pages_ouput_link_each_house_no_change   
    link_house = get_house_url(pages_url[i]) 
    try: 
        for j in link_house:     
            #input_link_each_house_no_change        
            data.extend(scrape_this(j))       
            #time.sleep(1)
    except:
        continue
    if i >= 49:
        break